In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("Groq_API_Key"):
    os.environ["GROQ_API_KEY"]=getpass.getpass("Enter API key for Groq:")

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [60]:
from typing import Any, Dict, List

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "Q1": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "2": {
        "Q2": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
    "3": {
        "Q3": "Multiple Choice Question",
        "Options": {
            "a": "choose this",
            "b": "choose this",
            "c": "choose this",
            "d": "choose this",
        },
        "Correct": "Correct Answer"
    },
}

In [8]:
template = """
Text:{text}
You are an expert AI MCQ maker. From the above given text, it is you job to \
create a quiz of {number} multiple choice questions for {subject} students in a {difficult} level.
Make Sure the questions do not get repeated and check all the questions to be vomforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

# can write this as well,
# template = """
# You are an AI teacher. Based on the following text, create 3 multiple-choice questions (MCQs) with the correct answers. 
# The questions should be relevant and based on the key concepts in the text. The format should be as follows:
# 1. Question: {question}
#    a) Option 1
#    b) Option 2
#    c) Option 3
#    d) Option 4
#    Correct answer: {correct_answer}

# Text: {text}

# Please generate the MCQs.
# """

In [44]:
quiz_Generation_Template = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
)

In [48]:
quiz_chain=LLMChain(llm=model, prompt=quiz_Generation_Template, output_key="quiz", verbose=True)

In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [43]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=TEMPLATE2
    )

In [39]:
review_chain=LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [50]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json", "difficult"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [51]:
file_path=r"C:\Users\divya\OneDrive\Project(MCQgen)\data.txt"

In [52]:
file_path

'C:\\Users\\divya\\OneDrive\\Project(MCQgen)\\data.txt'

In [53]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [54]:
print(TEXT)

The Tournament of Power was originally conceived by Zeno after witnessing the Tournament of Destroyers held by Beerus and Champa. After enjoying the tournament for its entertainment value and the spectacle of the participating fighters, Zeno determined that a larger competition involving all of the Universes would prove even more entertaining.

Later, after meeting the Future Zeno, Zeno had more or less forgotten about the tournament until reminded by an eager Goku. Having not seen the original martial arts competition, the Future Zeno was uncertain of the idea, leading to the need for the Zeno Expo, a smaller demonstration of a fighting competition.

It was at the Zeno Expo where the stakes of the Tournament of Power were set forth. Per the wishes of the two Zenos, the Grand Minister announced that the eight universes with the lowest fighter levels would be required to compete. The Universes that did not win the competition would immediately be erased from existence by the two Omni-Ki

In [98]:
# Serialize the Python dictionary into a JSON-formatted string
RESPONSE_JSON=json.dumps(RESPONSE_JSON)

In [99]:
NUMBER=5 
SUBJECT="biology"
TONE="Medium"

In [100]:
class LoggingHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.responses = []
    def on_chat_model_start(
        self, serialized: Dict[dict, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

        #Assuming response contains token information
        if hasattr(response, 'usage'):
            self.total_tokens += response.usage.get('total_tokens', 0)
            self.responses.append(response)

    def on_chain_start(
        self, serialized: Dict[dict, Any], inputs: Dict[dict, Any], **kwargs
    ) -> None:
        if serialized is not None:
            print(f"Chain {serialized.get('name')} started")
        else:
            print("Chain started with no serialized data")

    def on_chain_end(self, outputs: Dict[dict, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")
        print(f"Total tokens used: {self.total_tokens}")

In [101]:
logging_handler = LoggingHandler()

In [102]:
response = generate_evaluate_chain.invoke({"text": TEXT, "number": NUMBER, "subject": SUBJECT, "tone": TONE, "response_json": RESPONSE_JSON, "difficult": "Medium"}, callback_handler=logging_handler)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The Tournament of Power was originally conceived by Zeno after witnessing the Tournament of Destroyers held by Beerus and Champa. After enjoying the tournament for its entertainment value and the spectacle of the participating fighters, Zeno determined that a larger competition involving all of the Universes would prove even more entertaining.

Later, after meeting the Future Zeno, Zeno had more or less forgotten about the tournament until reminded by an eager Goku. Having not seen the original martial arts competition, the Future Zeno was uncertain of the idea, leading to the need for the Zeno Expo, a smaller demonstration of a fighting competition.

It was at the Zeno Expo where the stakes of the Tournament of Power were set forth. Per the wishes of the two Zenos, the Grand Minister announced that the eight universes with the lowest fighter levels would be required to compete. The Universes that did not win the competition would immediately be erased fr

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Here are 5 medium-level MCQs based on the provided text:

{
"1": {
"Q1": "Who conceived the idea of the Tournament of Power?",
"Options": {
"a": "Beerlus and Champa",
"b": "Zeno",
"c": "Grand Minister",
"d": "Future Zeno"
},
"Correct": "b"
},

"2": {
"Q2": "What was the original purpose of the Tournament of Power?",
"Options": {
"a": "To determine the strongest fighter in the multiverse",
"b": "To entertain the Omni-Kings",
"c": "To erase the weakest universes",
"d": "To reward the stron

In [103]:
print(response)

{'text': "The Tournament of Power was originally conceived by Zeno after witnessing the Tournament of Destroyers held by Beerus and Champa. After enjoying the tournament for its entertainment value and the spectacle of the participating fighters, Zeno determined that a larger competition involving all of the Universes would prove even more entertaining.\n\nLater, after meeting the Future Zeno, Zeno had more or less forgotten about the tournament until reminded by an eager Goku. Having not seen the original martial arts competition, the Future Zeno was uncertain of the idea, leading to the need for the Zeno Expo, a smaller demonstration of a fighting competition.\n\nIt was at the Zeno Expo where the stakes of the Tournament of Power were set forth. Per the wishes of the two Zenos, the Grand Minister announced that the eight universes with the lowest fighter levels would be required to compete. The Universes that did not win the competition would immediately be erased from existence by t

In [104]:
print(f"Total tokens used: {logging_handler.total_tokens}")

Total tokens used: 0


In [105]:
quiz = response.get("quiz")

In [106]:
print(quiz)

Here are 5 medium-level MCQs based on the provided text:

{
"1": {
"Q1": "Who conceived the idea of the Tournament of Power?",
"Options": {
"a": "Beerlus and Champa",
"b": "Zeno",
"c": "Grand Minister",
"d": "Future Zeno"
},
"Correct": "b"
},

"2": {
"Q2": "What was the original purpose of the Tournament of Power?",
"Options": {
"a": "To determine the strongest fighter in the multiverse",
"b": "To entertain the Omni-Kings",
"c": "To erase the weakest universes",
"d": "To reward the strongest warriors"
},
"Correct": "b"
},

"3": {
"Q3": "What were the stakes of the Tournament of Power?",
"Options": {
"a": "The winning universe would gain more power",
"b": "The losing universes would be erased",
"c": "The tournament would be cancelled",
"d": "The gods of destruction would be erased"
},
"Correct": "b"
},

"4": {
"Q4": "What was the format of the Tournament of Power?",
"Options": {
"a": "A one-on-one elimination tournament",
"b": "A survival-style battle royale",
"c": "A team-based competi

In [117]:
#lines = quiz.splitlines()
#quiz_without_first_line = "\n".join(lines[1:])

In [118]:
# 
quiz_new = pd.DataFrame(quiz.split("\n"), columns=["quiz"])

In [116]:

quiz_new.to_csv("quiz.csv", index=False)